In [1]:
# imports liberaries
import warnings 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import json 
import nltk
import spacy
import string
import unicodedata
from bs4 import BeautifulSoup
from textblob import TextBlob 
from nltk.stem import WordNetLemmatizer


from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate

from IPython import display 
display.set_matplotlib_formats('svg')
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#load the dataset
data = pd.read_csv("/content/gdrive/MyDrive/Reviews.csv")
data.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [4]:
data = data[1:1000]

In [5]:
data = data[['Score','Text']]
data.head(10)

,Score,Text
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...
5,4,I got a wild hair for taffy and ordered this f...
6,5,This saltwater taffy had great flavors and was...
7,5,This taffy is so good. It is very soft and ch...
8,5,Right now I'm mostly just sprouting this so my...
9,5,This is a very healthy dog food. Good for thei...
10,5,I don't know if it's the cactus or the tequila...


In [6]:
# check whether it's having null values or not
data.isnull().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of      Score   Text
1    False  False
2    False  False
3    False  False
4    False  False
5    False  False
..     ...    ...
995  False  False
996  False  False
997  False  False
998  False  False
999  False  False

[999 rows x 2 columns]>

In [7]:
# check unique categories
data['Score'].value_counts()

5    641
4    138
1     98
3     75
2     47
Name: Score, dtype: int64

In [8]:
# if rating is above 3 we will consider it as 1 else as 0.
data['Score'] = data['Score'].apply(lambda x: 1 if x > 3 else 0) # positive as 1 and negative as 0

### Text preprocessing

In [9]:
# 1. lowering words
data['Text'] = data['Text'].str.lower()
data.head()

,Score,Text
1,0,product arrived labeled as jumbo salted peanut...
2,1,this is a confection that has been around a fe...
3,0,if you are looking for the secret ingredient i...
4,1,great taffy at a great price. there was a wid...
5,1,i got a wild hair for taffy and ordered this f...


In [10]:
# 2. remove punctuation!
data['Text'] = data['Text'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', x))
data.head()

,Score,Text
1,0,product arrived labeled as jumbo salted peanut...
2,1,this is a confection that has been around a fe...
3,0,if you are looking for the secret ingredient i...
4,1,great taffy at a great price there was a wide...
5,1,i got a wild hair for taffy and ordered this f...


In [11]:
#3. remove stopwords
from spacy.lang.en.stop_words import STOP_WORDS
data['Text'] = data['Text'].apply(lambda x: " ".join([i for i in x.split() if i not in STOP_WORDS]))
data.head()

,Score,Text
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...
5,1,got wild hair taffy ordered pound bag taffy en...


In [12]:
#4. remove url and tags! 
data['Text'] = data['Text'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
data.head()

,Score,Text
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...
5,1,got wild hair taffy ordered pound bag taffy en...


In [13]:
# 5. remove html tags! 
data['Text'] = data['Text'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

data.head()

,Score,Text
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...
5,1,got wild hair taffy ordered pound bag taffy en...


In [14]:
# 7. remove emails
data['Text'] = data['Text'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)', '', x))
data.head()

,Score,Text
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...
5,1,got wild hair taffy ordered pound bag taffy en...


In [15]:
# 8. remove extra spaces
data['Text'] = data['Text'].apply(lambda x: " ".join(x.split()))
data.head()

,Score,Text
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...
5,1,got wild hair taffy ordered pound bag taffy en...


In [16]:
# 9. remove numbers
data['Text'] = data['Text'].str.replace('\d', '')
data.tail()

,Score,Text
995,1,black market hot sauce wonderful husband loves...
996,1,man salsa bomb different kinds meal owner cool...
997,1,sauce good like adding asian food egg noodles ...
998,0,hot like low star reviewer got suckered seeing...
999,0,admit sucker large quantity oz shopping hot s...


In [17]:
#@title
# 10. lemmatizer 
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
%time
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data['Text'] = data['Text'].apply(lambda text: lemmatize_words(text))
data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


,Score,Text
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection century light pillowy citrus gelati...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...
5,1,got wild hair taffy ordered pound bag taffy en...


## Sentiment analysis

In [18]:
data.head()

,Score,Text
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection century light pillowy citrus gelati...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...
5,1,got wild hair taffy ordered pound bag taffy en...


In [19]:
# NLTK a pretraind sentiment analyzer
nltk.download('vader_lexicon')
nlv = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [20]:
nlv.polarity_scores("The product was excellent")

{'compound': 0.5719, 'neg': 0.0, 'neu': 0.448, 'pos': 0.552}

In [21]:
nlv.polarity_scores("I liked this product but it is not good as the other one")

{'compound': -0.298, 'neg': 0.207, 'neu': 0.666, 'pos': 0.127}

In [22]:
data["Text"].apply(lambda x: x.upper())

1      PRODUCT ARRIVED LABELED JUMBO SALTED PEANUTSTH...
2      CONFECTION CENTURY LIGHT PILLOWY CITRUS GELATI...
3      LOOKING SECRET INGREDIENT ROBITUSSIN BELIEVE F...
4      GREAT TAFFY GREAT PRICE WIDE ASSORTMENT YUMMY ...
5      GOT WILD HAIR TAFFY ORDERED POUND BAG TAFFY EN...
                             ...                        
995    BLACK MARKET HOT SAUCE WONDERFUL HUSBAND LOVE ...
996    MAN SALSA BOMB DIFFERENT KIND MEAL OWNER COOL ...
997    SAUCE GOOD LIKE ADDING ASIAN FOOD EGG NOODLE G...
998    HOT LIKE LOW STAR REVIEWER GOT SUCKERED SEEING...
999    ADMIT SUCKER LARGE QUANTITY OZ SHOPPING HOT SA...
Name: Text, Length: 999, dtype: object

In [23]:
data["Text"][0:10].apply(lambda x: nlv.polarity_scores(x))

1     {'neg': 0.129, 'neu': 0.762, 'pos': 0.11, 'com...
2     {'neg': 0.134, 'neu': 0.568, 'pos': 0.297, 'co...
3     {'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'comp...
4     {'neg': 0.0, 'neu': 0.369, 'pos': 0.631, 'comp...
5     {'neg': 0.049, 'neu': 0.621, 'pos': 0.33, 'com...
6     {'neg': 0.062, 'neu': 0.523, 'pos': 0.415, 'co...
7     {'neg': 0.0, 'neu': 0.251, 'pos': 0.749, 'comp...
8     {'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'comp...
9     {'neg': 0.0, 'neu': 0.514, 'pos': 0.486, 'comp...
10    {'neg': 0.032, 'neu': 0.74, 'pos': 0.228, 'com...
Name: Text, dtype: object

In [24]:
data["Text"][0:10].apply(lambda x: nlv.polarity_scores(x)["compound"])

1    -0.1027
2     0.8624
3     0.4404
4     0.9468
5     0.9136
6     0.9099
7     0.9313
8     0.6369
9     0.8176
10    0.9528
Name: Text, dtype: float64

In [25]:
data["polarity_score"] = data["Text"].apply(lambda x: nlv.polarity_scores(x)["compound"])
data.head()

,Score,Text,polarity_score
1,0,product arrived labeled jumbo salted peanutsth...,-0.1027
2,1,confection century light pillowy citrus gelati...,0.8624
3,0,looking secret ingredient robitussin believe f...,0.4404
4,1,great taffy great price wide assortment yummy ...,0.9468
5,1,got wild hair taffy ordered pound bag taffy en...,0.9136


## Sentiment modeling

In [26]:
# Feature Engineering
data['Text'][0:10].apply(lambda x: 'pos' if nlv.polarity_scores(x)['compound'] > 0 else 'neg')


1     neg
2     pos
3     pos
4     pos
5     pos
6     pos
7     pos
8     pos
9     pos
10    pos
Name: Text, dtype: object

In [27]:
data["sentiment_label"] = data["Text"].apply(lambda x: "pos" if nlv.polarity_scores(x)["compound"] > 0 else "neg")
data.head(10)

,Score,Text,polarity_score,sentiment_label
1,0,product arrived labeled jumbo salted peanutsth...,-0.1027,neg
2,1,confection century light pillowy citrus gelati...,0.8624,pos
3,0,looking secret ingredient robitussin believe f...,0.4404,pos
4,1,great taffy great price wide assortment yummy ...,0.9468,pos
5,1,got wild hair taffy ordered pound bag taffy en...,0.9136,pos
6,1,saltwater taffy great flavor soft chewy candy ...,0.9099,pos
7,1,taffy good soft chewy flavor amazing definitel...,0.9313,pos
8,1,right im sprouting cat eat grass love rotate w...,0.6369,pos
9,1,healthy dog food good digestion good small pup...,0.8176,pos
10,1,dont know cactus tequila unique combination in...,0.9528,pos


In [28]:
data["sentiment_label"].value_counts()

pos    882
neg    117
Name: sentiment_label, dtype: int64

In [29]:
from sklearn.preprocessing import LabelEncoder
data["sentiment_label"] = LabelEncoder().fit_transform(data["sentiment_label"])

X = data["Text"]
y = data["sentiment_label"]

In [30]:
# Count Vectors
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_v = vectorizer.fit_transform(data['Text'])
vectorizer.get_feature_names()
X_v.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
#n-gram frequency
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X_n = vectorizer2.fit_transform(data['Text'])
vectorizer2.get_feature_names()
X_n.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word')
X_tf = vectorizer.fit_transform(data['Text'])
vectorizer.get_feature_names()
X_tf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
# n-gram tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2, 3))
X_n = vectorizer.fit_transform(data['Text'])
vectorizer.get_feature_names()
X_n.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Modeling

In [34]:
tf_idf_word_vectorizer = TfidfVectorizer()
X_tf_idf_word = tf_idf_word_vectorizer.fit_transform(X)

1.Logestic Regression

In [36]:
log_model = LogisticRegression().fit(X_tf_idf_word, y)

cross_val_score(log_model,
                X_tf_idf_word,
                y, scoring="accuracy",
                cv=5).mean()

new_comment = pd.Series("this product is great")
new_comment = pd.Series("look at that shit very bad")
new_comment = pd.Series("it was good but I am sure that it fits me")

new_comment = CountVectorizer().fit(X).transform(new_comment)
log_model.predict(new_comment)

array([1])

In [37]:
random_review = pd.Series(data["Text"].sample(1).values)
random_review

0    fair twin got gas horrible night screaming gas...
dtype: object

In [38]:
new_comment = CountVectorizer().fit(X).transform(random_review)
log_model.predict(new_comment)

array([0])

2. Random Forest

In [40]:
# Count Vectors
rf_model = RandomForestClassifier().fit(X_v, y)
cross_val_score(rf_model, X_v, y, cv=5, n_jobs=-1).mean()

0.8828844221105528

In [41]:
# TF-IDF Word-Level
rf_model = RandomForestClassifier().fit(X_tf_idf_word, y)
cross_val_score(rf_model, X_tf_idf_word, y, cv=5, n_jobs=-1).mean()

0.8828844221105528

In [43]:
# TF-IDF N-GRAM
rf_model = RandomForestClassifier().fit(X_n, y)
cross_val_score(rf_model, X_n, y, cv=5, n_jobs=-1).mean()

0.8828844221105528

In [46]:
#Hyper Parameter Optimization
rf_model = RandomForestClassifier(random_state=17)

rf_params = {"max_depth": [5, 8, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [2, 5, 8, 20],
             "n_estimators": [100, 200, 500]}

rf_best_grid = GridSearchCV(rf_model,
                            rf_params,
                            cv=5,
                            n_jobs=-1,
                            verbose=True).fit(X_v, y)

rf_best_grid.best_params_

rf_final = rf_model.set_params(**rf_best_grid.best_params_, random_state=17).fit(X_v, y)

cv_results = cross_validate(rf_final, X_v, y, cv=3, scoring=["accuracy", "f1", "roc_auc"])

Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [48]:
cv_results = cross_validate(rf_final, X_v, y, cv=3, scoring=["accuracy", "f1", "roc_auc"])